# 2️⃣ Re-Imaging Price Trends - 모델 학습 & 평가

**전제조건**: `1_image_generation.ipynb` 완료

**실행 내용**: CNN 모델 학습 → 포트폴리오 성과 평가 → 결과 시각화

In [ ]:
# 환경 설정
!pip install -r requirements.txt

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ReImaging_Price_Trends')
print(f"📁 현재 디렉토리: {os.getcwd()}")

# GPU 확인
import torch
print(f"🖥️ GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# 이미지 생성 완료 확인
required_dirs = ['train_I5R5', 'train_I20R20', 'train_I60R60', 'test_I5R5', 'test_I20R20', 'test_I60R60']
missing = []

for dir_name in required_dirs:
    metadata_file = f'images/{dir_name}/metadata.csv'
    if os.path.exists(metadata_file):
        print(f"✅ {dir_name}")
    else:
        print(f"❌ {dir_name}")
        missing.append(dir_name)

if missing:
    print(f"\n⚠️ 이미지 생성이 완료되지 않았습니다: {missing}")
    print("1_image_generation.ipynb를 먼저 실행하세요.")
else:
    print("\n✅ 모든 이미지 준비 완료!")

## 🧠 CNN 모델 학습

In [ ]:
# CNN5d 모델 학습
print("🧠 CNN5d 학습 시작...")
!python main.py --model CNN5d --image_days 5 --pred_days 5

# 메모리 정리
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()
print("✅ CNN5d 완료")

In [ ]:
# CNN20d 모델 학습
print("🧠 CNN20d 학습 시작...")
!python main.py --model CNN20d --image_days 20 --pred_days 20

# 메모리 정리
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()
print("✅ CNN20d 완료")

In [ ]:
# CNN60d 모델 학습
print("🧠 CNN60d 학습 시작...")
!python main.py --model CNN60d --image_days 60 --pred_days 60

# 메모리 정리
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()
print("✅ CNN60d 완료")

## 📈 포트폴리오 성과 평가

In [ ]:
# 모든 모델 평가
print("📈 포트폴리오 성과 평가 시작...\n")

print("1️⃣ CNN5d (Weekly Strategy)")
!python test.py --model CNN5d --image_days 5 --pred_days 5

print("\n2️⃣ CNN20d (Monthly Strategy)")
!python test.py --model CNN20d --image_days 20 --pred_days 20

print("\n3️⃣ CNN60d (Quarterly Strategy)")
!python test.py --model CNN60d --image_days 60 --pred_days 60

## 결과 시각화

In [ ]:
# 학습 곡선 시각화
import pandas as pd
import matplotlib.pyplot as plt

models = ['CNN5d_I5R5', 'CNN20d_I20R20', 'CNN60d_I60R60']
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Training Curves', fontsize=16, fontweight='bold')

for i, model in enumerate(models):
    log_file = f'logs/{model}.csv'
    if os.path.exists(log_file):
        df = pd.read_csv(log_file)
        
        # Loss
        axes[0, i].plot(df['train_loss'], 'b-', label='Train', linewidth=2)
        axes[0, i].plot(df['valid_loss'], 'r-', label='Valid', linewidth=2)
        axes[0, i].set_title(f'{model} - Loss')
        axes[0, i].legend()
        axes[0, i].grid(True, alpha=0.3)
        
        # Accuracy
        axes[1, i].plot(df['train_acc'], 'b-', label='Train', linewidth=2)
        axes[1, i].plot(df['valid_acc'], 'r-', label='Valid', linewidth=2)
        axes[1, i].set_title(f'{model} - Accuracy')
        axes[1, i].legend()
        axes[1, i].grid(True, alpha=0.3)
        
        final_acc = df['valid_acc'].iloc[-1]
        print(f"📊 {model}: 최종 검증 정확도 = {final_acc:.4f}")

plt.tight_layout()
plt.show()

In [ ]:
# 최종 결과 요약
print("🎯 Re-Imaging Price Trends - 실행 완료!")
print("=" * 50)

# 파일 크기 요약
if os.path.exists('models'):
    print("\n🧠 학습된 모델:")
    for file in sorted(os.listdir('models')):
        if file.endswith('.tar'):
            size_mb = os.path.getsize(f'models/{file}') / (1024**2)
            print(f"   ✅ {file} ({size_mb:.1f}MB)")

# 논문 성과와 비교
print("\n📚 논문 예상 성과 (비교 참조):")
print("   • Weekly (I5R5): H-L Sharpe = 7.15")
print("   • Monthly (I20R20): H-L Sharpe = 2.16") 
print("   • Quarterly (I60R60): H-L Sharpe = 0.37")

print("\n✅ 위의 포트폴리오 평가 결과와 비교해보세요!")